In [1]:
# Read and preprocess data
import preprocess as pre
import pandas as pd
df = pd.read_csv('./Data/4017105.csv')
df = pre.convertTime(df)
df = pre.calculateETA(df)
df = pre.encodeTime(df, 5)
df.drop('Heading', axis=1, inplace=True)
print(df.isnull().sum())

Day_sin           0
Day_cos           0
Hour              0
Minute            0
Second            0
Vehicle_id        0
Speed             0
Passenger_load    0
Next_stop         0
Latitude          0
Longitude         0
Traffic_speed     0
Temperature       0
Windspeed         0
Precipitation     0
Humidity          0
Visibility        0
Stop_distance     0
ETA               0
dtype: int64


In [2]:
# Perform one hot encoding of categorical variables
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(handle_unknown='ignore')
encoder_df = pd.DataFrame(encoder.fit_transform(df[['Vehicle_id', 'Next_stop']]).toarray())
df = df.join(encoder_df)
df.drop('Vehicle_id', axis=1, inplace=True)
df.drop('Next_stop', axis=1, inplace=True)
# df = df.dropna()

In [3]:
# Split data into dependent and independent variables
X = df.iloc[:, df.columns != 'ETA'].values
y = df.iloc[:, df.columns == 'ETA'].values

In [4]:
# Split data into training and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, shuffle=False) # Allocates last 20% of data as test set

In [5]:
# Scale and standardize data
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
sc_y = StandardScaler()
X_train  = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)
# y_train = sc_y.fit_transform(y_train)
# y_test = sc_y.transform(y_test)

In [8]:
# Train SVR model on the training set
from sklearn.svm import SVR
regressor = SVR(kernel='rbf', C=10, gamma=0.1, epsilon=0.1)
regressor.fit(X_train, y_train)

/Users/bliau/.pyenv/versions/3.11.3/envs/RuTransloc/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVR(C=10, gamma=0.1)

In [9]:
# Predicting the test set results
y_pred = regressor.predict(X_test)
print(y_pred)

[ 52.86471093  88.16270285  93.14779678 ...  96.35313647 132.96448727
 111.85665334]


In [10]:
# Calculate accuracy
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
r2 = r2_score(y_test, y_pred)
print('R-squared score:', r2)
mae = mean_absolute_error(y_test, y_pred)
print('Mean absolute error:', mae)
mse = mean_squared_error(y_test, y_pred)
print('Mean squared error:', mse)

R-squared score: 0.13952994449898615
Mean absolute error: 152.55121391291718
Mean squared error: 54245.0154341342


In [11]:
import plotly.express as px
import pandas as pd

fig = px.line(y_test)
fig2 = px.line(y_pred)
fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(step="all")
        ])
    )
)
fig2.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(step="all")
        ])
    )
)
fig.show()
fig2.show()